# GBDT - LightGBM

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
import os
import numpy as np
import pandas as pd
import lightgbm
import pickle
import json
import functools
from os import listdir
from os.path import isfile, join
from nilib import *

In [9]:
def train_gradient_boosting_baseline(train_file, valid_file, test_file, output_model_file, drop_cols=None):
    
    exp = pd.DataFrame(columns=['num_trees', 'learning_rate', 'num_leaves', 'best_round', 'metric', 'filename'])
    
    # load train/valid/test
    train, valid, test, cat_fx = load_atk_train_valid_test(train_file, valid_file, test_file)
    
    assert "instance_id" not in train.columns.values, "Wrong training set file for GBDT"

    # dropping some of the columns
    if drop_cols is not None:
        print ("Dropping columns:", drop_cols)
        train.drop(columns=drop_cols, inplace=True)
        valid.drop(columns=drop_cols, inplace=True)
        test.drop(columns=drop_cols, inplace=True)
    
    
    # get index of categorical features 
    cat_fx = np.where(train.columns.isin(cat_fx))[0]
    cat_fx = list([int(x) for x in cat_fx])  
    print("CatFX:", train.columns.values[cat_fx])
    

    for num_trees in [100]:
        for learning_rate in [0.05]: #[0.01, 0.05]:
            for num_leaves in [24]: #[16, 24]:
                # datasets
                lgbm_train = lightgbm.Dataset(data=train.iloc[:,:-1].values, 
                                              label=train.iloc[:,-1].values,
                                              categorical_feature = cat_fx)

                lgbm_valid = lightgbm.Dataset(data=valid.iloc[:,:-1].values, 
                                              label=valid.iloc[:,-1].values,
                                              categorical_feature = cat_fx)

                # run train
                lgbm_params = { 'learning_rate': learning_rate, 
                                'num_leaves': num_leaves,
                                'objective': 'regression'
                              } 
                lgbm_info = {}
                lgbm_model = lightgbm.train(lgbm_params, lgbm_train, 
                                            num_boost_round = num_trees,
#                                             fobj            = optimize_log_loss, 
#                                             feval           = avg_log_loss,
                                            evals_result    = lgbm_info,
                                            valid_sets      = [lgbm_train, lgbm_valid], 
                                            valid_names     = ['train', 'valid'],
                                            verbose_eval    = 50)
                
                best_valid_iter = np.argmin(lgbm_info['valid']['l2'])
                
                model_file_name = "{:s}_T{:d}_S{:04d}_L{:d}_R{:d}.model".format(output_model_file,
                                                                            num_trees,
                                                                            int(learning_rate*1000),
                                                                            num_leaves,
                                                                            best_valid_iter + 1
                                                                           )
                
                # update experimental results
                exp = exp.append({'num_trees': num_trees, 
                                  'learning_rate':learning_rate,
                                  'num_leaves':num_leaves, 
                                  'best_round':best_valid_iter+1, 
                                  'metric':lgbm_info['valid']['l2'][best_valid_iter],
                                  'filename':model_file_name},
                                 ignore_index=True)
                
        
                lgbm_model.save_model(model_file_name)
                print("Model saved to", model_file_name)

    
    return exp

# WINE Dataset

In [10]:
DATASET_NAME="wine"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)

In [13]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

Pre-processing original files...
Loading: ../data/wine/train.csv.bz2
Loading: ../data/wine/valid.csv.bz2
Loading: ../data/wine/test.csv.bz2
Train/Valid/Test sizes: (4547, 13) (650, 13) (1300, 13)
Train/Valid/Test split: 0.70 0.10 0.20
CatFX: []
Train/Valid/Test sizes: (3898, 13) (1299, 13) (1300, 13)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.cat.bz2
CatFX: []
[50]	train's l2: 0.387329	valid's l2: 0.455517
[100]	train's l2: 0.312616	valid's l2: 0.427861
Model saved to ../out/models/wine/std-gbdt_wine_T100_S0050_L24_R100.model
  num_trees  learning_rate num_leaves best_round    metric  \
0       100           0.05         24        100  0.427861   

                                            filename  
0  ../out/models/wine/std-gbdt_wine_T100_S0050_L2...  
best model is: ../out/models/wine/std-gbdt_wine_T100_S0050_L24_R100.model


In [14]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=["alcohol", "residual_sugar", "volatile_acidity"]
                                              )  

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

Pre-processing original files...
Loading: ../data/wine/train.csv.bz2
Loading: ../data/wine/valid.csv.bz2
Loading: ../data/wine/test.csv.bz2
Train/Valid/Test sizes: (4547, 13) (650, 13) (1300, 13)
Train/Valid/Test split: 0.70 0.10 0.20
CatFX: []
Train/Valid/Test sizes: (3898, 13) (1299, 13) (1300, 13)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.cat.bz2
Dropping columns: ['alcohol', 'residual_sugar', 'volatile_acidity']
CatFX: []
[50]	train's l2: 0.433816	valid's l2: 0.513682
[100]	train's l2: 0.360147	valid's l2: 0.486498
Model saved to ../out/models/wine/red-gbdt_wine_T100_S0050_L24_R100.model
  num_trees  learning_rate num_leaves best_round    metric  \
0       100           0.05         24        100  0.486498   

                                            filename  
0  ../out/models/wine/red-gbdt_wine_T100_S0050_L2...  
best model is: ../out/models/wine/red-gbdt_wine_T100_S0050_L24_R100.model


# CENSUS Dataset

In [ ]:
DATASET_NAME="census"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)

In [ ]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=['workclass','marital_status',
                                                          'occupation', 'education_num',
                                                          'hours_per_week','capital_gain' ]
                                              )

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

# SPAM

In [15]:
DATASET_NAME="spam"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)

In [17]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

Pre-processing original files...
Loading: ../data/spam/train.csv.bz2
Loading: ../data/spam/valid.csv.bz2
Loading: ../data/spam/test.csv.bz2
Train/Valid/Test sizes: (2760, 58) (921, 58) (920, 58)
Train/Valid/Test split: 0.60 0.20 0.20
CatFX: []
Train/Valid/Test sizes: (2760, 58) (920, 58) (921, 58)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.cat.bz2
CatFX: []
[50]	train's l2: 0.0348443	valid's l2: 0.0558493
[100]	train's l2: 0.0233714	valid's l2: 0.0484396
Model saved to ../out/models/spam/std-gbdt_spam_T100_S0050_L24_R99.model
  num_trees  learning_rate num_leaves best_round    metric  \
0       100           0.05         24         99  0.048388   

                                            filename  
0  ../out/models/spam/std-gbdt_spam_T100_S0050_L2...  
best model is: ../out/models/spam/std-gbdt_spam_T100_S0050_L24_R99.model


In [21]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=['char_freq_!', 'word_freq_remove',
                                                          'char_freq_$', 'capital_run_length_average',
                                                          'capital_run_length_total', 'word_freq_hp' ]
                                              )

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

Pre-processing original files...
Loading: ../data/spam/train.csv.bz2
Loading: ../data/spam/valid.csv.bz2
Loading: ../data/spam/test.csv.bz2
Train/Valid/Test sizes: (2760, 58) (921, 58) (920, 58)
Train/Valid/Test split: 0.60 0.20 0.20
CatFX: []
Train/Valid/Test sizes: (2760, 58) (920, 58) (921, 58)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.cat.bz2
Dropping columns: ['char_freq_!', 'word_freq_remove', 'char_freq_$', 'capital_run_length_average', 'capital_run_length_total', 'word_freq_hp']
CatFX: []
[50]	train's l2: 0.0434418	valid's l2: 0.0624735
[100]	train's l2: 0.0310334	valid's l2: 0.0554811
Model saved to ../out/models/spam/red-gbdt_spam_T100_S0050_L24_R92.model
  num_trees  learning_rate num_leaves best_round    metric  \
0       100           0.05         24         92  0.055465   

                                            filename  
0  ../out/models/spam/red-gbdt_spam_T100_S0050_L2...  
best model is: ../out/models/spam/red-gbdt_spam_T100_S0050_L24_R92.mo

In [27]:
TRAIN, _, _, _ = load_atk_train_valid_test(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME)
print (TRAIN.columns)

Pre-processing original files...
Loading: ../data/spam/train.csv.bz2
Loading: ../data/spam/valid.csv.bz2
Loading: ../data/spam/test.csv.bz2
Train/Valid/Test sizes: (2760, 58) (921, 58) (920, 58)
Train/Valid/Test split: 0.60 0.20 0.20
CatFX: []
Train/Valid/Test sizes: (2760, 58) (920, 58) (921, 58)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.cat.bz2
Index(['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d',
       'word_freq_our', 'word_freq_over', 'word_freq_remove',
       'word_freq_internet', 'word_freq_order', 'word_freq_mail',
       'word_freq_receive', 'word_freq_will', 'word_freq_people',
       'word_freq_report', 'word_freq_addresses', 'word_freq_free',
       'word_freq_business', 'word_freq_email', 'word_freq_you',
       'word_freq_credit', 'word_freq_your', 'word_freq_font', 'word_freq_000',
       'word_freq_money', 'word_freq_hp', 'word_freq_hpl', 'word_freq_george',
       'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word

In [58]:
TRAIN[TRAIN['word_freq_hp']>=1]['word_freq_hp'].describe()

count    417.000000
mean       3.370048
std        3.132073
min        1.000000
25%        1.610000
50%        2.320000
75%        3.520000
max       20.830000
Name: word_freq_hp, dtype: float64

In [26]:
###------------_####
def print_fx_imp(model, colnames):
    fx_uses = model.feature_importance(importance_type='split')
    fx_gain = model.feature_importance(importance_type='gain')

    for i,f in enumerate(np.argsort(fx_gain)[::-1]):
        print ("{:2d} {:20s} {:.3f} {:4d}".format(i, colnames[f], fx_gain[f], fx_uses[f]))

        

print(" -- GDBT --")    
gbdt = lightgbm.Booster(model_file="../out/models/spam/std-gbdt_spam_T100_S0050_L24_R99.model")
print(gbdt.num_trees())
print_fx_imp(gbdt, TRAIN.columns)

print(" -- Reduced GDBT --")    
redf = lightgbm.Booster(model_file="../out/models/spam/red-gbdt_spam_T100_S0050_L24_R92.model")
print(redf.num_trees())
print_fx_imp(redf, TRAIN.drop(columns=['char_freq_!', 'word_freq_remove',
                                                          'char_freq_$', 'capital_run_length_average',
                                                          'capital_run_length_total', 'word_freq_hp' ]).columns
            )


# print(" -- Adv. Boosting --")    
# advb = lightgbm.Booster(model_file="../out/models/census/adv-boosting_census_B30_T100_S0050_L24_R100.model")
# print(advb.num_trees())
# print_fx_imp(advb, TRAIN.columns)


 -- GDBT --
100
 0 char_freq_!          1675.403  179
 1 word_freq_remove     898.772   68
 2 char_freq_$          884.417   72
 3 capital_run_length_average 417.196  172
 4 capital_run_length_total 340.651  163
 5 word_freq_hp         336.482  104
 6 word_freq_free       180.480   90
 7 word_freq_money      175.816   25
 8 capital_run_length_longest 171.871  158
 9 word_freq_your       158.627  106
10 word_freq_our        151.961  110
11 word_freq_edu        128.443   74
12 word_freq_george     92.992   75
13 word_freq_you        72.602  124
14 word_freq_business   64.007   47
15 word_freq_000        41.320   34
16 word_freq_re         32.453   54
17 char_freq_(          31.598   76
18 word_freq_will       26.902   70
19 word_freq_1999       26.525   35
20 word_freq_hpl        20.583    6
21 word_freq_internet   20.143   23
22 word_freq_meeting    18.177   39
23 word_freq_technology 13.427   16
24 char_freq_#          13.078   27
25 word_freq_over       12.347   44
26 word_freq_font  